In [1]:
from awscrt import io, mqtt, auth, http
from awsiot import mqtt_connection_builder
from pathlib import Path
import time as t
import json

FILE_PATH = Path("../../Data/Processed/energy_comsuption.csv")
ENDPOINT = "a4lvmadg3sako-ats.iot.us-east-1.amazonaws.com"
CLIENT_ID = "house_energy_comsuption"
PATH_TO_CERTIFICATE = "house_energy_comsuption.cert.pem"
PATH_TO_PRIVATE_KEY = "house_energy_comsuption.private.key"
PATH_TO_AMAZON_ROOT_CA_1 = "root-CA.crt"

TOPIC = "house/comsuption"


In [2]:
def conexao_aws():
    event_loop_group = io.EventLoopGroup(1)
    host_resolver = io.DefaultHostResolver(event_loop_group)
    client_bootstrap = io.ClientBootstrap(event_loop_group, host_resolver)
    connection_builder = mqtt_connection_builder.mtls_from_path(
        endpoint=ENDPOINT,
        cert_filepath=PATH_TO_CERTIFICATE,
        pri_key_filepath=PATH_TO_PRIVATE_KEY,
        ca_filepath=PATH_TO_AMAZON_ROOT_CA_1,
        client_bootstrap=client_bootstrap,
        client_id=CLIENT_ID,
        clean_session=False,
        keep_alive_secs=6)
    
    connection_future = connection_builder.connect()
    connection_future.result()
    return connection_builder, connection_future

In [3]:
def lendo_entradas(linha_atual=0):
    with open(FILE_PATH, "r") as dado:
        dado.seek(linha_atual)
        for linha in dado:
            yield linha

In [8]:
if __name__ == "__main__":
    read_line=0
    connection = conexao_aws()
    for idx, line in enumerate(lendo_entradas(linha_atual=read_line)):
        data = line.split(",")
        message = {"DATA": data[0],
                  "COMSUPTION_KWH": data[1],
                  "COST": data[2],
                  "MOUNTH_GOAL": data[3].rstrip('\n'),
                  }
        print(message)
        connection.publish(topic=TOPIC,
                          payload=json.dumps(message),
                          qos=QoS.AT_LEAST_ONCE)
        t.sleep(.5)
    disconnect_future = connection.disconnect()
    disconnect_future.result()
    t.sleep(4)

{'DATA': '2016-10-22', 'COMSUPTION_KWH': '4.5200000000000005', 'COST': '0.6000000000000002', 'MOUNTH_GOAL': '300.0'}


AwsCrtError: AWS_ERROR_MQTT_UNEXPECTED_HANGUP: The connection was closed unexpectedly.